In [1]:
from ddpg import ReplayBuffer, DDPGAgent
from fsae.envs import *
import time

pybullet build time: Apr 10 2024 13:56:47


In [2]:
env = RandomTrackEnv(render_mode=None, seed=0)

argv[0]=
argv[0]=


In [3]:
# Initialize the agent, replay buffer, and environment
state_dim = 12 # Dimension of the state space
action_dim = 2 # Dimension of the action space
hidden_dim = 256
max_action = (1,0.6) # Maximum value of the action
num_episodes = 1000
max_steps = 500
batch_size = 128

replay_buffer = ReplayBuffer(buffer_size=50000, state_dim=state_dim, action_dim=action_dim)
replay_buffer.load_from_csv("replay_buffers/replayBuffer_teleop_test.csv")
agent = DDPGAgent(state_dim, action_dim, hidden_dim, replay_buffer, max_action)

if replay_buffer.size > batch_size:
    print("Replay buffer size after load: ", replay_buffer.size, " vs Batch Size: ", batch_size)
    agent.train(batch_size)

rewards = []
avg_rewards = []
best_reward  = 0

# Training loop
for episode in range(num_episodes):
    state = env.reset(seed=episode)
    episode_reward = 0

    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        if done: 
            reward -= 0.5
        
        replay_buffer.add(state, action, reward, next_state, done)
        state = next_state
        episode_reward += reward

        if replay_buffer.size > batch_size:
            agent.train(batch_size)

        if done:
            break
    
    rewards.append(episode_reward)

    if (episode % 10 == 0):
        avg_reward = np.mean(np.asarray(rewards))
        print(f"Average Episode Reward: {avg_reward}")
        rewards.clear()
        avg_rewards.append(avg_reward)
        if avg_reward > best_reward:
            agent.save_weights()
            best_reward = avg_reward

agent.save_weights(best=False)


Replay buffer size after load:  5537  vs Batch Size:  128
Average Episode Reward: -1.693944340102678
Average Episode Reward: 0.644396338835515
Average Episode Reward: 7.853827148097823
Average Episode Reward: 9.392630518819573
Average Episode Reward: 8.823770636627893
Average Episode Reward: 8.640582509763986
Average Episode Reward: 4.137006891953139
Average Episode Reward: 9.909630295338756
Average Episode Reward: 6.688111533792127
Average Episode Reward: 10.025118343443292
Average Episode Reward: 9.457149733703837
Average Episode Reward: 8.554348353643281
Average Episode Reward: 13.018723908725374
Average Episode Reward: 13.198163002957324
Average Episode Reward: 9.6859246166967
Average Episode Reward: 5.3544860610885365
Average Episode Reward: 6.284100620848362
Average Episode Reward: 9.238972887098972
Average Episode Reward: 11.399988115024803
Average Episode Reward: 8.228044584331652
Average Episode Reward: 4.83950382163657
Average Episode Reward: 5.138843662505842
Average Episode

KeyboardInterrupt: 

In [4]:
agent.save_weights(best=False)


In [ ]:
replay_buffer.save_as_csv("replayBuffer_train_test.csv")

In [7]:
try:
    env.close()
except:
    pass    
env = RandomTrackEnv(render_mode='tp_camera', seed=0)



argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3090/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.171.04
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.171.04
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3090/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [11]:
done = False
state = env.reset(seed=4)
agent.load_weights()

while not done:
    action = agent.get_action(state)
    next_state, reward, done, _ = env.step(action)
    print(reward)

0.2637833876602505
0.16083727998001168
0.17515989475168403
0.1691778336034403
0.16127845114205974
0.15196942530656532
0.13880068842563076
0.11811161348623767
0.09009490381997454
0.054289855931912934
0.008192499587276414
-0.03860371049113631
-0.08736608672028434
-0.14738489976207503
-0.22223007957019114
-0.31327372868716474
-0.02142284210754586
0.05046516297867942
-0.015462809580798043
6.325198821177658e-05
-0.0018939770167392123
0.008396187037121194
0.004475055606903422
0.004292370185067895
-0.0008981559455429888


KeyboardInterrupt: 